In [1]:
import ee
import requests
import os
# Initialize the Earth Engine API
ee.Initialize(project='ts-rishimnb')


In [2]:
import rasterio
import numpy as np
from rasterio.enums import Resampling
from osgeo import gdal


def get_data(start_date, end_date):
    """
    Downloads the NDVI band from the MODIS image collection as a GeoTIFF file for the given date range.

    Parameters:
    start_date (str): The start date in the format 'YYYY-MM-DD'.
    end_date (str): The end date in the format 'YYYY-MM-DD'.
    """
    # Define the area of interest (AOI) using coordinates (hardcoded)
    bbox = ee.Geometry.Rectangle([78.00405826, 16.93264351, 79.04971836, 17.90150706])
    
    # Choose the MODIS image collection and filter by date and bounds
    image_collection = ee.ImageCollection("MODIS/061/MOD11A1") \
        .filterBounds(bbox) \
        .filterDate(start_date, end_date)
    
    # Calculate the mean image over the date range
    img = image_collection.mean()
    
    # Generate the download URL for the NDVI band
    url = img.getDownloadUrl({
        'bands': ['LST_Night_1km'],
        'region': bbox,
        'scale': 1000,
        'format': 'GEO_TIFF'
    })
    
    # Download the file
    response = requests.get(url)
    with open('nlst_2.tif', 'wb') as fd:
        fd.write(response.content)
    

    input_image = 'nlst_2.tif'

    # Define the target bounding box (xmin, ymin, xmax, ymax)
    # These are the coordinates of the bounding box you want to crop to.
    target_bbox = [78.00405826, 16.93264351, 79.04971836, 17.90150706]

    # Path for the output cropped image
    output_image = 'cropped_nlst_image.tif'

    # Open the image using GDAL
    dataset = gdal.Open(input_image)

    # Crop the image using gdal.Warp
    gdal.Warp(output_image, dataset, outputBounds=target_bbox)

    # Close the dataset
    dataset = None

    geo_tiff_path = 'cropped_nlst_image.tif'
    height_factor=1/3.5
    width_factor=1/7
    with rasterio.open(geo_tiff_path) as dataset:
     data = dataset.read(
        out_shape=(
            dataset.count,
            int(dataset.height * height_factor),
            int(dataset.width * width_factor)
        ),
        resampling=Resampling.average
    )

    # Scale image transform
    transform = dataset.transform * dataset.transform.scale(
        (dataset.width / data.shape[-1]),
        (dataset.height / data.shape[-2])
    )

    # Define the new metadata for the output file
    new_meta = dataset.meta.copy()
    new_meta.update({
        'height': data.shape[1],
        'width': data.shape[2],
        'transform': transform
    })

    # Save the resampled image
    with rasterio.open('resampled_image_nlst.tif', 'w', **new_meta) as dst:
        dst.write(data)

    new_path='resampled_image_nlst.tif'
    with rasterio.open(new_path) as dataset:
       band1=dataset.read()
       return band1[0]

In [6]:
x=get_data('2023-09-01', '2023-09-30')
print(np.shape(x))
print(x)


(30, 16)
[[    0.             0.             0.             0.
      0.             0.             0.             0.
      0.             0.             0.          2707.27969349
   9493.25478927  1661.21264368     0.             0.        ]
 [    0.             0.             0.             0.
      0.             0.             0.             0.
      0.             0.             0.          5307.40996169
  14720.08812261  4676.86206897     0.             0.        ]
 [    0.             0.             0.             0.
      0.             0.             0.             0.
      0.             0.             0.          1016.07662835
  12694.08429119 12002.04597701   225.30268199     0.        ]
 [    0.             0.             0.             0.
      0.             0.          2828.1532567      0.
      0.             0.             0.             0.
   5385.91570881 10184.42911877  3726.59770115     0.        ]
 [    0.             0.             0.             0.
      0.     

In [4]:
# months = {
#     1: ("January", 31),
#     2: ("February", 28),
#     3: ("March", 31),
#     4: ("April", 30),
#     5: ("May", 31),
#     6: ("June", 30),
#     7: ("July", 31),
#     8: ("August", 31),
#     9: ("September", 30),
#     10: ("October", 31),
#     11: ("November", 30),
#     12: ("December", 31)
# }

# for year in range(2018,2024):
#     for i in range(1,13):
#         output_dir=fr"C:\Users\rishi\ml_projects\climate\{year}_data\nlst_scaled"
#         os.makedirs(output_dir, exist_ok=True)
#         month_name, end_day = months[i]
#         start=f"{year}-{i}-1"
#         end = f"{year}-{i}-{end_day}"
#         arr=get_data(start,end)
#         path=os.path.join(output_dir, f'{month_name}_NLST.csv')
#         np.savetxt(path, arr, delimiter=',')

In [5]:
# import pandas as pd

# df= pd.read_csv(r"C:\Users\rishi\ml_projects\climate\2018_data\nlst\January_NLST.csv")
# df